# <hr style="clear: both" />

# Running Spark in YARN-client mode

This notebook demonstrates how to set up a SparkContext that uses SURFsara's Hadoop cluster: [YARN resourcemanager](http://head05.hathi.surfsara.nl:8088/cluster) (note you will need to be authenticated via kerberos on your machine to visit the resourcemanager link) for executors.

First initialize kerberos via a Jupyter terminal. 
In the terminal execute: <BR>
<i>kinit -k -t data/robertop.keytab robertop@CUA.SURFSARA.NL</i><BR>
Print your credentials:


In [1]:
! klist

Ticket cache: FILE:/tmp/krb5cc_1001
Default principal: robertop@CUA.SURFSARA.NL

Valid starting       Expires              Service principal
04/12/2016 13:42:29  04/13/2016 13:42:29  krbtgt/CUA.SURFSARA.NL@CUA.SURFSARA.NL
	renew until 04/12/2016 13:42:29


In [2]:
print sc

# <hr style="clear: both" />

# Now you can run your code

Pick a clustering algorithm (name of the file that provides a classify(x,y [,threshold]) function)

In [3]:
execfile('../spark-scripts/conventions.py')
execfile('../spark-scripts/splitCluster2.py')
#execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/implicitPlaylistAlgoFunctions.py')
execfile('../spark-scripts/implicitPlaylistAlgoMain.py')

CLUSTER_ALGO = 'jaccardBase'

execfile('../spark-scripts/' + CLUSTER_ALGO + '.py')


# Reading the conf file

In [4]:
import json
import copy

BASE_PATH = "/mnt/space/mattia"

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'SenzaRipetizioni_1'
conf['split']['split'] = conf['split']['name']
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = BASE_PATH + '/' + CLUSTER_ALGO + '.split/SenzaRipetizioni_1'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = BASE_PATH
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 1
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False
conf['split']["prop"] = {'reclistSize': conf['split']['reclistSize']}
conf['split']['type'] = list
conf['split']['out'] = BASE_PATH + '/' + CLUSTER_ALGO + '.split'
conf['split']['location'] = '30Mdataset/relations/sessions'

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = CLUSTER_ALGO + '.split'
conf['general']['bucketName'] = BASE_PATH
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = CLUSTER_ALGO
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']["sessionJaccardShrinkage"] = 5
conf['algo']['props']["clusterSimilarityThreshold"] = 0.1
conf['algo']['props']["expDecayFactor"] = 0.7
# ****** END EXAMPLE ****************#

# <hr style="clear: both" />
# Pick the list of songs ad create clusters


In [136]:
import json
import string

def my_replace_punct(x):
    ret = ""
    for i in x:
        if i == '+':
            ret += ' '
        else:
            ret += i
    return ret

tracksRDD = sc.textFile(BASE_PATH + '/30Mdataset/entities/tracks.idomaar.gz')
tracksRDD = tracksRDD.map(lambda x: x.split('\t')).map(lambda x: (x[1], json.loads(x[3])['name'].split('/') ) )
tracksRDD = tracksRDD.map(lambda x: (x[0], " ".join( (x[1][0], x[1][2]) ) ))
tracksRDD = tracksRDD.map(lambda x : (x[0], my_replace_punct(x[1])))
tracksRDD = tracksRDD.map(lambda x: (x[0], tokenize_song(x[1]), x[1]))


sampleRDD = tracksRDD.take(50000)
sampleRDD = sc.parallelize(sampleRDD)

tracksIdsRDD = sampleRDD.map(lambda x: (x[0], [x[0]]))

sampleRDD.take(3)


[(u'0',
  [u'000003',
   u'music',
   u'instructor',
   u'dj27s',
   u'rock',
   u'da',
   u'house',
   u'c382e28988c386e28988c38a01',
   u'dj',
   u'maxpulemet',
   u'vs',
   u'bomfunk',
   u'mc27s',
   u'electro',
   u'breakdance',
   u'party',
   u'1',
   u'5b20005d',
   u'cd',
   u'onec382e28988c386e28988c38a'],
  u'000003 Music Instructor Dj%27s Rock Da House %C3%82%E2%89%88%C3%86%E2%89%88%C3%8A01 - Dj Max-Pulemet Vs. Bomfunk Mc%27s - Electro Breakdance party 1 %5B2000%5D = CD ONE%C3%82%E2%89%88%C3%86%E2%89%88%C3%8A'),
 (u'1',
  [u'0001', u'd0a2d0b5d0bad181d182'],
  u'00-01 %D0%A2%D0%B5%D0%BA%D1%81%D1%82'),
 (u'2', [u'0005', u'overkill', u'overkill'], u'0005. Overkill Overkill')]

# <hr style="clear: both" />

Reduce the quantity of data by building RDD {word -> songs}.
For each word keep only couples of songs that match.


In [137]:
#Build an RDD with ('word' -> (id, name))
wordsRDD = sampleRDD.flatMap(lambda x: [(i, (x[0], x[2])) for i in x[1]] )
wordsRDD.take(3)

#Group by 'word' and keep only the ones with more then 1 song
wordsRDD = wordsRDD.groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 1)

#Compute a cartesian product for each list of songs with a common word
def filtered_cartesian(x):
    equal_couples = set()
    for i in range(len(x[1])):
        a = x[1][i]
        id_a = x[1][i][0]
        name_a = x[1][i][1]
        
        for j in range(i):
            b = x[1][j]
            id_b = x[1][j][0]
            name_b = x[1][j][1]
            if id_a != id_b:
                if classify(name_a, name_b):
                    equal_couples.add((a,b))
                    
    return (x[0], tuple(equal_couples))

coupleRDD = wordsRDD.map(filtered_cartesian).filter(lambda x: len(x[1]) > 1)
coupleRDD.take(3)

[(u'and',
  (((u'3081', u'112 Peaches And Cream'),
    (u'2997', u'112 112 - Peaches and Cream')),
   ((u'2859', u'10 Years ... And All The Other Colors'),
    (u'2858', u'10 Years ...And All the Other Colors')),
   ((u'43005',
     u'69 Jazzfunkclassics - Ladies and gentlemen 69 Jazzfunkclassics - Ladies and gentlemen'),
    (u'43004', u'69 Jazzfunkclassics Ladies and Gentlemen')),
   ((u'19408', u'%3F and the Mysterians Can%27t Get Enough Of You, Baby'),
    (u'19407', u'%3F and the Mysterians Can%27t Get Enough of You Baby')),
   ((u'38964', u'%5Bunknown%5D Shut Up And Drive - Rihanna'),
    (u'38420', u'%5Bunknown%5D Rihanna - Shut Up And Drive')),
   ((u'47512',
     u'9GOATS BLACK OUT Melancholy and the loneliness -%22%E6%86%82%E9%AC%B1%E3%81%A8%E5%AD%A4%E7%8B%AC%22 re-mix-'),
    (u'47511',
     u'9GOATS BLACK OUT Melancholy and the loneliness- %22%E6%86%82%E9%AC%B1%E3%81%A8%E5%AD%A4%E7%8B%AC%22 re-mix -')),
   ((u'39817', u'%5Bunknown%5D The 12 Days of Christmas - Kunt and the 

# <hr style="clear: both" />
Flip the dataset and map each song to the couples it belongs to. 
Group by key and for each song you have a cluster!


In [ ]:
#Flatmap the list of couples
flattedCoupleRDD = coupleRDD.flatMap(lambda x: [i for i in x[1]])
#For each couple, for each song, yield song->couple
flattedCoupleRDD = flattedCoupleRDD.flatMap(lambda x: ((i[0], (x[0], x[1]) )for i in x) )

#Group by key (song). Each song has now one cluster
def merge_couples(x, y):
    return list(set(x) | set(y))

songClusterRDD = flattedCoupleRDD.reduceByKey(merge_couples)
songClusterRDD.take(30)

# <hr style="clear: both" />
Complete the clusters with all the other songs. 
Then we need to map cluster to songs to have new IDs.

In [134]:
#In this way we obtain a complete RDD with song -> group of songs
def reduce_to_biggest(x, y):
    bigger = x if len(x) > len(y) else y
    result = sorted(bigger)
    return result
           
unionRDD = songClusterRDD.union(tracksIdsRDD).reduceByKey(reduce_to_biggest)
unionRDD.take(100)


[(u'3610', [u'3610', u'3611']),
 (u'151', [u'151']),
 (u'605', [u'605']),
 (u'3922', [u'3922']),
 (u'3948', [u'3948']),
 (u'3773', [u'3773']),
 (u'4260', [u'4260']),
 (u'2617', [u'2617']),
 (u'740', [u'740']),
 (u'1304', [u'1304']),
 (u'2594', [u'2594']),
 (u'1227', [u'1227']),
 (u'883', [u'883']),
 (u'2778', [u'2778']),
 (u'1142', [u'1142']),
 (u'1087', [u'1087']),
 (u'3827', [u'3827']),
 (u'3759', [u'3759']),
 (u'2734', [u'2734']),
 (u'4303', [u'4303']),
 (u'1788', [u'1788']),
 (u'1164', [u'1164']),
 (u'1322', [u'1322']),
 (u'4365', [u'4365']),
 (u'623', [u'623']),
 (u'4189', [u'4189']),
 (u'1128', [u'1128']),
 (u'4329', [u'4329']),
 (u'2712', [u'2712']),
 (u'2903', [u'2903']),
 (u'3593', [u'3593']),
 (u'1348', [u'1348']),
 (u'3904', [u'3904']),
 (u'2860', [u'2860']),
 (u'3715', [u'3715']),
 (u'4080', [u'4080']),
 (u'216', [u'216']),
 (u'591', [u'591']),
 (u'2477', [u'2477']),
 (u'2965', [u'2965']),
 (u'458', [u'458']),
 (u'2929', [u'2929']),
 (u'3905', [u'3905']),
 (u'150', [u'150']

In [119]:
#
clusterSongsRDD = unionRDD.map(lambda x: (x[1], x[0])).groupByKey().mapValues(list)
clusterSongsRDD = clusterSongsRDD.zipWithIndex().map(lambda x: (x[1], x[0][1]))
clusterSongsRDD.saveAsTextFile(BASE_PATH + '/clusters/' + CLUSTER_ALGO)

In [ ]:
splitter(conf)

In [ ]:
from os import path
basePath = path.join(conf['general']['bucketName'], conf['general']['clientname'])
splitPath = path.join(basePath, conf['split']['name'])

clusterSimList = [0.1]
sessionJaccardShrinkageList = [5]
expDecayList = [0.7]

for exclude in [True]:
    conf['split']['excludeAlreadyListenedTest'] = str(exclude)
    #conf['split']['name'] = 'giroCompletoTestMultipleConfs_exclude%s' % exclude
    #splitter(conf)
    train, test = loadDataset(conf)
    train.cache()
    test.cache()
    
    for sessionJaccardShrinkage in sessionJaccardShrinkageList:
        conf['algo']['props']["sessionJaccardShrinkage"] = sessionJaccardShrinkage
        
        for clusterSim in clusterSimList:
            conf['algo']['props']["clusterSimilarityThreshold"] = clusterSim
            
            playlists = extractImplicitPlaylists(train, conf).cache()
            
            for expDecay in expDecayList:
                conf['algo']['props']["expDecayFactor"] = expDecay
                conf['algo']['name'] = CLUSTER_ALGO + '_ImplicitPlaylist_shk_%d_clustSim_%.3f_decay_%.3f' % \
                    (sessionJaccardShrinkage, clusterSim, expDecay )

                recJsonRDD = executeImplicitPlaylistAlgo(playlists, test, conf)
                try:
                    saveRecommendations(conf, recJsonRDD, overwrite=True)
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'
                except:
                    print 'Error in saving recommndations'
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'